# <I>Preprecossing des collections:</I>
Utilisation de NLP (Natural Language Processing) par sa bib python NLTK (Natural Language Tool Kit).
<I><strong>Le NLP combine la linguistique computationnelle "règle basée sur la modélisation du langage humain "avec des modèles statistiques et d'apprentissage automatique pour permettre aux ordinateurs et aux appareils numériques, comprendre et générer du texte et de la parole.</strong></I>
Les techniques NLP utilisées sont:
## <I>1.Tokinisation:</I>
processus de conversion d'une séquence de texte en parties plus petites, appelées jetons (tokens).Ces jetons peuvent être aussi petits que des caractères ou aussi longs que des mots. La principale raison pour laquelle ce processus est important est qu'il facilite l'analyse.
["Chatbots", "are", "helpful"] = ["C", "h", "a", "t", "b", "o", "t", "s", " ", "a", "r", "e", " ", "h", "e", "l", "p", "f", "u", "l"].
### Types de Tokinization :
Tokenisation de mots, Tokenisation de caractères et Tokenisation de sous-mots.
## <I>2.Suppression des mots vides (stopwords):</I>
Les mots-clés sont les plus courants mots dans n'importe quelle langue naturelle. Aux fins de l'analyse des données textuelles etde la création de modèles de PNL, ces mots d'arrêt peuvent ne pas ajouter beaucoup de valeur à la signification du document.
Généralement, les mots les plus couramment utilisés dans un texte francais sont “ le ”, “ est ”, “ dans ”, “ pour ” où “, ” quand... et dans un texte anglais "and “, "or", "who", "which",......La suppression des mots d'arrêt peut potentiellement aider à améliorer les performances car il reste moins de jetons significatifs. Cela pourrait augmenter la précision de la classification.
## <I>3.Lemmatizer:</I>
Est une technique de pré-traitement de texte utilisée dans les modèles de traitement du langage naturel (NLP) pour décomposer un mot à sa signification fondamentale afin d'identifier les similitudes. Par exemple, un algorithme de lemmatisation réduirait le mot mieux à sa racine, ou lemme(forme de dictionnaire).Par exemple, vous pouvez vous attendre à un algorithme de lemmatisation pour mapper “runs,” “running,” et “ran” au lemme, “run.”

In [4]:
import pymongo as mongo

client = mongo.MongoClient("mongodb://localhost:27017/")
db = client["admin"]
db = client["admin"]

collection_questions = db["questions"]
collection_answers = db["answers"]

questions_clean = db["questions_clean"]
answers_clean = db["answers_clean"]

#Vider les collections
questions_clean.delete_many({})
answers_clean.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [5]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [6]:
questions = list(collection_questions.find())

def preprocess_text(question):
    # Convertir la question en minuscules
    question = question.lower()
    exp = re.compile(r'<.*?>')
    question = re.sub(exp, '', question)
    # Supprimer la ponctuation
    question = re.sub(r'[^\w\s]', '', question)
    
    # Tokenization
    tokens = word_tokenize(question)
    
    # Supprimer les mots vides (stop words)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Reconstruire la question à partir des tokens lemmatisés
    processed_question = ' '.join(tokens)
    
    return processed_question

# Appliquer le prétraitement à toutes les questions
preprocessed_data = []
for item in questions:
    preprocessed_item = {}
    for key, value in item.items():
        if key != "link":
            # Appliquer le prétraitement sur les valeurs de type String
            if isinstance(value, str):
                preprocessed_item[key] = preprocess_text(value)
            else:
                preprocessed_item[key] = value
        else:
            preprocessed_item[key] = value

        if key == "answers":
            for answer in value:
                for field_key, field_value in answer.items():
                    if isinstance(field_value, str):
                        answer[field_key] = preprocess_text(field_value)
        elif isinstance(value, str):
            item[key] = preprocess_text(value)

    preprocessed_data.append(preprocessed_item)

# Enregistrer les données prétraitées dans une nouvelle collection MongoDB
db["questions_clean"].insert_many(preprocessed_data)
print("Prétraitement terminé. Les données ont été enregistrées dans la collection questions_clean de MongoDB.")

Prétraitement terminé. Les données ont été enregistrées dans la collection questions_clean de MongoDB.


In [7]:
from html.parser import HTMLParser
import re
import string

answers = list(collection_answers.find())

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def clean_answer_text(answer):
    # Supprimer les balises HTML
    cleaned_text = remove_html_tags(answer)
    
    # Supprimer les caractères de contrôle et les espaces superflus
    cleaned_text = re.sub(r'[\r\n\t]+', ' ', cleaned_text)
    cleaned_text = cleaned_text.strip()
    
    # Supprimer la ponctuation (optionnel)
    cleaned_text = cleaned_text.translate(str.maketrans('', '', string.punctuation))
    
    return cleaned_text


preprocessed_data = []
for item in answers:
    preprocessed_item = {}
    for key, value in item.items():
        if key != "link":
            if isinstance(value, str):
                preprocessed_item[key] = clean_answer_text(value)
            else:
                preprocessed_item[key] = value

        else:
            preprocessed_item[key] = value

    preprocessed_data.append(preprocessed_item)

# Enregistrer les données prétraitées dans une nouvelle collection MongoDB
db["answers_clean"].insert_many(preprocessed_data)
print("Prétraitement terminé. Les données ont été enregistrées dans la collection answers_clean de MongoDB.")

Prétraitement terminé. Les données ont été enregistrées dans la collection answers_clean de MongoDB.
